In [5]:
import tensorflow as tf
tf.keras.__version__

'2.4.0'

# 使用Keras搭建神经网络


在Keras中，MNIST数据集以一组四个Numpy数组的形式预先加载

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

`“train_images”和“train_labels”构成了“training set”，即模型将从中学习的数据。然后，模型将在计算机上进行测试
"test set", `test_images` 和 `test_labels`. 我们的图像被编码为Numpy数组，标签只是一个数字数组，范围很广 
从0到9。图像和标签之间有一对一的对应关系。

我们来看看训练数据:

In [7]:
train_images.shape

(60000, 28, 28)

In [8]:
len(train_labels)

60000

In [9]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Let's have a look at the test data:

In [10]:
test_images.shape

(10000, 28, 28)

In [11]:
len(test_labels)

10000

In [12]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

我们的工作流程如下：首先，我们将用训练数据展示我们的神经网络， `train_images` 和 `train_labels`.  
网络将学习如何将图像和标签联系起来。最后，我们将要求网络生成“train_images”的预测，并验证这些预测是否与“train_labels”中的标签匹配。 

让我们来构建我们的网络——再次提醒大家，您还不应该了解这个示例的所有内容.

In [13]:
network = tf.keras.models.Sequential()
network.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(tf.keras.layers.Dense(10, activation='softmax'))


神经网络的核心组成部分是“层”，一个数据处理模块，你可以把它想象成数据的“过滤器”。一些数据以更有用的形式出现。准确地说，层从输入的数据中提取表示——希望表示对当前问题更有意义。大多数深度学习实际上是将简单的层链接在一起，从而实现一种渐进式的“数据提炼”。一个深度学习模型就像一个数据处理的筛子，由一系列越来越精细的数据过滤器——“层”组成。

在这里，我们的网络由两个密集层组成，这两个层是密集连接（也称为“完全连接”）的神经层。第二层（也是最后一层）是一个10路“softmax”层，这意味着它将返回一个包含10个概率分数（总和为1）的数组。每个分数将是当前数字图像属于我们的10个数字类之一的概率。

为了使我们的网络为培训做好准备，我们还需要选择三件事，作为“编译”步骤的一部分：

* 损失函数：是指网络将如何衡量它在训练数据方面做得有多好，从而如何将自己引导到正确的方向。
* 优化器：这是一种机制，通过这种机制，网络将根据所看到的数据及其丢失功能进行自我更新。
* 培训和测试期间要监控的指标。这里我们只关心准确度（正确分类的图像的分数） 

损失函数和优化器的确切用途将在接下来的两章中阐明.

In [14]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


在训练之前，我们将对数据进行预处理，将其重塑为网络所期望的形状，并对其进行缩放，使所有值都在[0，1]间隔内。例如，以前我们的训练图像存储在uint8类型的形状数组（60000，28，28）中，其值在[0255]间隔内。我们将其转换为一个float32数组，数组的形状为（60000，28*28），值介于0和1之间。

In [15]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

我们还需要对标签进行分类编码，这一步我们将在第3章中解释:

In [16]:
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

我们现在已经准备好训练我们的网络，在Keras中，这是通过调用网络的“fit”方法来完成的：
我们将模型“拟合”到它的训练数据中.

In [17]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 7ms/step - loss: 0.2569 - accuracy: 0.9260
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.1046 - accuracy: 0.9688
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0680 - accuracy: 0.9801
Epoch 4/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0500 - accuracy: 0.9851
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0372 - accuracy: 0.9890


在训练期间显示两个量：网络在训练数据上的“丢失”和网络在训练数据上的准确性 

训练数据的准确率很快达到0.989（98.9%）。现在，让我们检查一下我们的模型在测试集上的性能:

In [18]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 917us/step - loss: 0.0677 - accuracy: 0.9793


In [19]:
print('test_acc:', test_acc)

test_acc: 0.9793000221252441



我们的测试集准确率是97.8%——这比训练集的准确率要低很多。训练精度和测试精度之间的这种差距是“过度拟合”的一个例子，即机器学习模型在新数据上的表现往往比在训练数据上的表现更差。过度装修将是第三章的中心话题。
我们的第一个例子到此结束——您刚刚看到我们如何构建和训练一个神经网络来对手写数字进行分类，只需不到20行Python代码。在下一章中，我们将详细讨论我们刚刚预演的每一个动人的片段，并阐明幕后真正发生的事情。您将了解“张量”，即进入网络的数据存储对象，了解张量运算（由哪些层组成），以及梯度下降，这使我们的网络能够从其训练示例中学习。